## Processing a text file

In [ ]:
import nltk

In [ ]:
f = open('romeo.txt', 'r')  # open romeo.txt in read mode

In [ ]:
rom = f.read()   # use f.read to read file data and store it in variable rom

In [ ]:
print(rom[:1000])   # print the first 1000 characters

In [ ]:
romList = rom.split()
print(romList)

In [ ]:
romText = nltk.Text(romList)
romText.concordance('she')

In [ ]:
nltk.download("punkt")

In [ ]:
rWords = nltk.word_tokenize(rom)
rWords

In [ ]:
rSent = nltk.sent_tokenize(rom)

In [ ]:
nltk.download("stopwords")

In [ ]:
import string
string.punctuation

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
useless = stopwords + list(string.punctuation)
useless

In [ ]:
bag = []
for w in rWords:
    if w not in useless:
        bag.append(w)
print(bag)

In [ ]:
bag = [w for w in rWords if w not in useless]    # one-line code

## Getting text from an html file

In [ ]:
import urllib.request

In [ ]:
url = "http://news.bbc.co.uk/2/hi/health/2284783.stm"

In [ ]:
html = urllib.request.urlopen(url).read()    # this was the missing line

In [ ]:
html[:1000]

In [ ]:
from bs4 import BeautifulSoup   # BeautifulSoup is a library for cleaning html file 

In [ ]:
raw = BeautifulSoup(html).get_text()
raw[:1000]

In [ ]:
tokens = nltk.word_tokenize(raw)
tokens

In [ ]:
text = nltk.Text(tokens)
text

In [ ]:
text.concordance('gene')

## Movie Reviews Dataset
NLTK corpus with 200 text files, each is a review of a movie.  They are split in a neg folder
for the negative reviews and a pos folder for the positive reviews 

In [ ]:
nltk.download("movie_reviews")

In [ ]:
nltk.download()   # to download nltk data interactively

In [ ]:
from nltk.corpus import movie_reviews

The `fileids` method provided by all the datasets in `nltk.corpus` gives access to a list of all the files available.

In [ ]:
len(movie_reviews.fileids())

In [ ]:
movie_reviews.fileids()[:5]

fileids can also filter the available files based on their category, which is the name of the subfolders they are located in. Therefore we can have lists of positive and negative reviews separately.

In [ ]:
negative_fileids = movie_reviews.fileids('neg')
positive_fileids = movie_reviews.fileids('pos')
len(negative_fileids), len(positive_fileids)

We can inspect one of the reviews using the raw method of movie_reviews, each file is split into sentences, the curators of this dataset also removed from each review from any direct mention of the rating of the movie.

In [ ]:
print(movie_reviews.raw(fileids=positive_fileids[0]))   

The movie_reviews corpus already has direct access to tokenized text with the words method:

In [ ]:
movie_reviews.words(fileids=positive_fileids[0])

In [ ]:
all_words = movie_reviews.words()
len(all_words)                        # how many words are in the entire movie reviews corpus?

In [ ]:
filtered_words = [w for w in movie_reviews.words() if w not in useless] 

In [ ]:
from collections import Counter
word_counter = Counter(filtered_words)

In [ ]:
most_common_words = word_counter.most_common()[:10]
most_common_words

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
sorted_word_counts = sorted(list(word_counter.values()), reverse=True)

plt.loglog(sorted_word_counts)
plt.ylabel("Freq")
plt.xlabel("Word Rank");

In [ ]:
plt.hist(sorted_word_counts, bins=50);

In [ ]:
plt.hist(sorted_word_counts, bins=50, log=True);

## Train a Classifier for Sentiment Analysis

Using our `build_bag_of_words_features` function we can build separately the negative and positive features.
Basically for each of the 1000 negative and for the 1000 positive review, we create one dictionary of the words and we associate the label "neg" and "pos" to it.

In [ ]:
def build_bag_of_words_filtered(words):
    return {
        word:1 for word in words \
        if not word in useless}

In [ ]:
negative_features = [
    (build_bag_of_words_filtered(movie_reviews.words(fileids=[f])), 'neg') \
    for f in negative_fileids
]

In [ ]:
print(negative_features[3])

In [ ]:
positive_features = [
    (build_bag_of_words_filtered(movie_reviews.words(fileids=[f])), 'pos') \
    for f in positive_fileids
]

In [ ]:
print(positive_features[6])

One of the simplest supervised machine learning classifiers is the Naive Bayes Classifier, we will train it on 80% of the data to learn what words are generally associated with positive or with negative reviews.

In [ ]:
from nltk.classify import NaiveBayesClassifier

In [ ]:
split = 800

In [ ]:
sentiment_classifier = NaiveBayesClassifier.train(positive_features[:split]+negative_features[:split])

In [ ]:
nltk.classify.util.accuracy(sentiment_classifier, positive_features[:split]+negative_features[:split])*100

The accuracy above is mostly a check that nothing went very wrong in the training, the real measure of accuracy is on the remaining 20% of the data that wasn't used in training, the test data:

In [ ]:
nltk.classify.util.accuracy(sentiment_classifier, positive_features[split:]+negative_features[split:])*100

Accuracy here is around 70% which is pretty good for such a simple model if we consider that the estimated accuracy for a person is about 80%. We can finally print the most informative features, i.e. the words that mostly identify a positive or a negative review:

In [ ]:
sentiment_classifier.show_most_informative_features()